---
<div align="center">

# Lung Cancer Classification
</div>

---

In [8]:
import numpy as np
np.int = np.int32
import pandas as pd
import matplotlib.pyplot as plt
import radiomics
import pylidc as pl

In [2]:
# pl.__version__

In [3]:
# !pip install pylidc

In [4]:
help(dir)

Help on built-in function dir in module builtins:

dir(...)
    dir([object]) -> list of strings
    
    If called without an argument, return the names in the current scope.
    Else, return an alphabetized list of names comprising (some of) the attributes
    of the given object, and of attributes reachable from it.
    If the object supplies a method named __dir__, it will be used; otherwise
    the default dir() logic is used and returns:
      for a module object: the module's attributes.
      for a class object:  its attributes, and recursively the attributes
        of its bases.
      for any other object: its attributes, its class's attributes, and
        recursively the attributes of its class's base classes.



---
## Data Exploration and Extraction
---

In [16]:
# Checking the amount of patients available
print(f"There are {np.unique(np.array([scan.patient_id for scan in pl.query(pl.Scan).all()])).size} Total Patients")

There are 1010 Total Patients


In [5]:
# Checking the amount of scans available
print(f"There are {len(pl.query(pl.Scan).all())} Scans available")

There are 1018 Scans available


In [6]:
# Checking the amount of Annotations available
print(f"There are {len(pl.query(pl.Annotation).all())} Annotation available")

There are 6859 Annotation available


---
### Patient Nodules Visualization
---

Let's select the first patient scan available and visualize how his nodules data look like


In [9]:
# Fetching the first available scan and the correpondent patient
scan = pl.query(pl.Scan).first()
patientID = scan.patient_id
patientScanNodules = scan.cluster_annotations()
print(f"The Patient {patientID}'s Scan has {len(patientScanNodules)} Nodules")

The Patient LIDC-IDRI-0078's Scan has 4 Nodules


In [10]:
patientScanNodulesAnnotations = scan.annotations

In [11]:
ann = patientScanNodulesAnnotations[0]
ann.visualize_in_scan()

Loading dicom files ... This may take a moment.


RuntimeError: Couldn't find DICOM files for Scan(id=1,patient_id=LIDC-IDRI-0078) in C:\Users\Gongas\LIDC-IDRI\LIDC-IDRI-0078

---
### Annotation Class Analysis
---

In [5]:
ann = pl.query(pl.Annotation).first()
ann.print_formatted_feature_table()

Feature              Meaning                    # 
-                    -                          - 
Subtlety           | Obvious                  | 5 
Internalstructure  | Soft Tissue              | 1 
Calcification      | Absent                   | 6 
Sphericity         | Ovoid                    | 3 
Margin             | Near Sharp               | 4 
Lobulation         | No Lobulation            | 1 
Spiculation        | No Spiculation           | 1 
Texture            | Solid                    | 5 
Malignancy         | Indeterminate            | 3 


In [6]:
# Inspect all the attributes inside an Annotation Object
annotationParams = [param for param, paramValue in pl.Annotation.__dict__.items() if param[0] != '_' and not callable(paramValue)]
# annotationParams

Let's take a look on the first available Annotation

In [10]:
pl.query(pl.Annotation).first()

Annotation(id=1,scan_id=1)

---
### Scan Class Analysis
---

In [7]:
# Inspect all the attributes inside an Scan Object
scanParams = [param for param, paramValue in pl.Scan.__dict__.items() if param[0] != '_' and not callable(paramValue)]
# scanParams